<a href="https://colab.research.google.com/github/gsilsbe/Missouri_Reservoirs/blob/main/MU_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!pip install earthengine-api
#!pip install folium
#!pip install geopandas

## Interactive map <a class="anchor" id="interactive-map"></a>

The [`folium`](https://python-visualization.github.io/folium/)library can be used to display `ee.Image` objects on an interactive [Leaflet](https://leafletjs.com/) map. Folium has no default method for handling tiles from Earth Engine, so one must be definedand added to the `folium.Map` module before use.

The following cells provide an example of adding a method for handing Earth Engine tiles and using it to display an elevation model to a Leaflet map.

In [9]:
# Import packages
import folium
from folium import plugins
import numpy as np
import geopandas as gpd
import random

### Add custom basemaps <a class="anchor" id="add-basemaps"></a>

In [10]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

## Load geojson files

In [15]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

#DNR = gpd.read_file("MO_water.geojson", driver='GeoJSON')
MU = gpd.read_file("MU.geojson", driver='GeoJSON')

# Sort alphabetically
#DNR = DNR.sort_values('WATER_BODY')
MU = MU.sort_values('MULakeNumber')

ERROR:fiona._env:MU.geojson: No such file or directory


DriverError: MU.geojson: No such file or directory

## Plot Waterbodies, Watershed and Ecoregions with ancillary GEE Data

In [ ]:
my_map = folium.Map(location=[37.8978,-90.6275], zoom_start=13, control_scale = True, tiles='Stamen Terrain')

basemaps['Google Satellite Hybrid'].add_to(my_map)

# DNR Waterbody Polygons
folium.Choropleth(
    geo_data=MU,
    name='Choropleth',
    data=MU,
    columns=['MULakeNumber', 'volumeM3'],
    key_on="feature.properties.MULakeNumber",
    fill_color= "YlGnBu",
    threshold_scale=custom_scale,
    legend_name='waterBody',
    fill_opacity=0.5,
    line_opacity=1,
    smooth_factor=0
).add_to(my_map)

# Eco Regions
for x in MU.index:
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    color = '#'+''.join(color).upper()
    MU.at[x, 'color'] = color

for _, r in MU.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    fillColor = r['color']
    color = r['color']
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, name=r["waterBody"], show=True,
                           style_function=lambda x, fillColor=fillColor,
                           color=color: {
                                          "fillColor": color,
                                          "color": color,
                                          "fill_opacity":0.5,
                                      })
    folium.Popup(r["waterBody"]).add_to(geo_j)
    geo_j.add_to(my_map)


# Water body icons
for i in range(0,len(MU.MULakeNumber)):
    folium.Marker(
        location=[MU.iloc[i]['waterBodyLatitude'], MU.iloc[i]['waterBodyLongitude']],
         icon=folium.map.Icon(color='green'),
        popup=MU.iloc[i]['waterBody'],
   ).add_to(my_map)

my_map.add_child(folium.LayerControl())
my_map.add_child(folium.LatLngPopup())
my_map

## Land use classification by watershed

In [ ]:
MU

,waterBodyID,WBID,MULakeNumber,waterBody,WATER_BODY_DNR,waterBodyLatitude,waterBodyLongitude,ecoregionID,damHeightM,volumeM3,surfAreaHA,watershedAreaHA,geometry
0,1,7177,1,Glover,Glover Spring Lake,38.8476,-91.8876,2.0,10.1,460088.0,7.8,1489.0,"POLYGON ((-10229712.493 4700028.648, -10229708..."
1,2,7249,2,Pinnacle,Pinnacle Lake,38.8258,-91.4197,2.0,14.9,3066431.0,38.4,1900.1,"POLYGON ((-10177606.898 4698462.905, -10177589..."
2,3,7004,3,Bowling Green (East),Bowling Green Reservoir,39.3416,-91.1530,2.0,22.3,1739207.0,8.9,337.0,"POLYGON ((-10146805.741 4769470.627, -10146816..."
3,4,7003,4,Bowling Green (West),Old Lake,39.3423,-91.1611,2.0,18.6,1303788.0,18.2,10.3,"POLYGON ((-10147936.714 4771052.299, -10147932..."
4,5,7049,5,Lincoln,Lake Lincoln,39.0203,-90.9217,2.0,21.0,1894625.0,12.8,404.2,"POLYGON ((-10120917.366 4726230.452, -10120915..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,370,7253,461,See Tal,See Tal Lake,38.6881,-91.4368,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-10178590.917 4676728.886, -10178604..."
348,371,0,462,Tilles Park,None,38.6222,-90.3636,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-90.363 38.623, -90.363 38.623, -90...."
349,372,7181,463,McKee Pond,Lake Champetra,38.6710,-92.2592,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-10139972.658 4646030.547, -10139993..."
350,373,0,464,Cedar (Randolph),Rothwell Lake,39.3925,-92.4234,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-92.423 39.392, -92.423 39.392, -92...."
